<a href="https://colab.research.google.com/github/AiaUrq/Lung-Cancer-Mortality-EDA-and-Machine-Learning/blob/main/Random_forest_LC_aus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc, recall_score, precision_score, f1_score

In [ ]:
from google.colab import files

# This will prompt you to select a file
uploaded = files.upload()


Saving lung_cancer_mortality_sample.csv to lung_cancer_mortality_sample.csv


In [ ]:
import pandas as pd
df = pd.read_csv("lung_cancer_mortality_sample.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'lung_cancer_mortality_sample.csv'

In [ ]:
data = df.drop(["id","country", "cirrhosis","diagnosis_date","beginning_of_treatment_date", "treatment_type", "end_treatment_date" ], axis = 1)

In [ ]:
def _transform_column_into_dummies(dataframe, name_of_column):
    dummies = pd.get_dummies(dataframe[name_of_column], prefix=f"{name_of_column} = ")

    dataframe = pd.concat([dataframe, dummies], axis=1)

    dataframe = dataframe.drop([name_of_column], axis=1)

    return dataframe


In [ ]:
data = _transform_column_into_dummies(data, "smoking_status")

In [ ]:
data = _transform_column_into_dummies(data, "cancer_stage")


In [ ]:
data = _transform_column_into_dummies(data, "gender")


In [ ]:
data = _transform_column_into_dummies(data, "asthma")

In [ ]:
data = _transform_column_into_dummies(data, "family_history")

In [ ]:
#data = _transform_column_into_dummies(data, "treatment_type")

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162500 entries, 0 to 162499
Data columns (total 20 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   age                               162500 non-null  int64  
 1   bmi                               162500 non-null  float64
 2   cholesterol_level                 162500 non-null  int64  
 3   hypertension                      162500 non-null  int64  
 4   other_cancer                      162500 non-null  int64  
 5   survived                          162500 non-null  int64  
 6   smoking_status = _Current Smoker  162500 non-null  bool   
 7   smoking_status = _Former Smoker   162500 non-null  bool   
 8   smoking_status = _Never Smoked    162500 non-null  bool   
 9   smoking_status = _Passive Smoker  162500 non-null  bool   
 10  cancer_stage = _Stage I           162500 non-null  bool   
 11  cancer_stage = _Stage II          162500 non-null  b

In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop(["survived"], axis = 1)
y=data["survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#I had to run grid search multiple times on different hyperparameters so it didn't crash
#from sklearn.model_selection import GridSearchCV
#param_grid = {
#    'n_estimators': [400, 500]
#}

#rf = RandomForestClassifier()
#grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='precision')
#grid_search.fit(X_train, y_train)
#print(grid_search.best_params_)

Grid search version for 'best' precision

In [ ]:
rf = RandomForestClassifier(n_estimators=400,
        max_depth=30,
        min_samples_split=5,
        min_samples_leaf=2,
        max_features='sqrt',
        bootstrap=True,
        oob_score=True,
        n_jobs=-1,
        random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=30, min_samples_leaf=2, min_samples_split=5,
                       n_estimators=400, n_jobs=-1, oob_score=True,
                       random_state=42)

In [ ]:
y_pred_rf = rf.predict(X_test)


In [ ]:
accuracy = accuracy_score(y_test, y_pred_rf)
accuracy

0.7791692307692307

In [ ]:
precision = precision_score(y_test, y_pred_rf)
precision

0.23507462686567165

In [ ]:
recall = recall_score(y_test, y_pred_rf)
recall

0.008955223880597015

In [ ]:
f1_score = f1_score(y_test, y_pred_rf)
f1_score

0.01725318362316856

In [ ]:
features = data.drop(["survived"], axis = 1)
feature_names = [f"feature {i}" for i in range(features.shape[1])]
feature_importances = pd.Series(rf.feature_importances_, index=features.columns).sort_values(ascending=False)

In [ ]:
feature_importances

bmi                                 0.324474
cholesterol_level                   0.300921
age                                 0.253222
hypertension                        0.016720
other_cancer                        0.010281
cancer_stage = _Stage IV            0.007634
cancer_stage = _Stage II            0.007460
cancer_stage = _Stage III           0.007393
cancer_stage = _Stage I             0.007387
asthma = _1                         0.007257
asthma = _0                         0.007216
gender = _Male                      0.006958
gender = _Female                    0.006898
smoking_status = _Never Smoked      0.006897
smoking_status = _Current Smoker    0.006559
smoking_status = _Passive Smoker    0.006491
smoking_status = _Former Smoker     0.006042
family_history = _Yes               0.005104
family_history = _No                0.005089
dtype: float64

In [ ]:
rf = RandomForestClassifier(n_estimators=400,
        max_depth=30,
        min_samples_split=10,
        min_samples_leaf=5,
        max_features='sqrt',
        bootstrap=True,
        oob_score=True,
        n_jobs=-1,
        random_state=42, class_weight = "balanced" )
rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=30,
                       min_samples_leaf=5, min_samples_split=10,
                       n_estimators=400, n_jobs=-1, oob_score=True,
                       random_state=42)

In [ ]:
y_pred = rf.predict(X_test)

NameError: name 'rf' is not defined

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

NameError: name 'y_pred' is not defined

In [ ]:
precision = precision_score(y_test, y_pred)
precision

0.22151054176674673

In [ ]:
recall = recall_score(y_test, y_pred)
recall

0.11798152096659559

logistic regression


In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(class_weight="balanced")
lr.fit(X_train, y_train)

lr_predictions = lr.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, lr_predictions)
accuracy

0.500523076923077

In [ ]:
precision = precision_score(y_test, lr_predictions)
precision

0.21544363321371118